In [1]:
import pandas as pd

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['No'].tolist()

总数量：2


[1, 2]

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

import json

from bs4 import BeautifulSoup
from lxml import etree

url = input_.loc[a, 'Url']

dict_param = {}
list_param = url.split('?')[1].split('&') if '?' in url else []
for param in list_param:
    dict_param[param.split('=')[0]] = param.split('=')[1]

dict_param.pop('_pgn', '')
dict_param['orig_cvip'] = 'true'
dict_param['_ipg'] = '240'

if url.startswith('https://www.ebay.com/'):
    dict_param['_ul'] = 'US'
    dict_param['_stpos'] = '91710'
elif url.startswith('https://www.ebay.de/'):
    dict_param['_ul'] = 'DE'
    dict_param['_stpos'] = '10115'
elif url.startswith('https://www.ebay.co.uk/'):
    dict_param['_ul'] = 'GB'
    dict_param['_stpos'] = 'BS81QU'
elif url.startswith('https://www.ebay.com.au/'):
    dict_param['_ul'] = 'AU'
    dict_param['_stpos'] = '2019'
elif url.startswith('https://www.ebay.ca/'):
    dict_param['_ul'] = 'CA'
    dict_param['_stpos'] = 'M5S2E8'

url = url.split('?')[0] + '?'
for key, value in dict_param.items():
    url += f'{key}={value}&'

list_item_number, i = [], 1
while True:
    b = 0
    while True:
        b += 1
        print(b, i)
        try:
            resp = requests.get(f'''{url}_pgn={i}''',
                                headers=get_header(ua=False),
                                proxies=get_proxy(),
                                timeout=(10, 10))
            
            if resp.status_code == 200:
                break
        except KeyboardInterrupt:
            break
        except:
            continue

    # = = = = = = = = = = = = = = =

    soup = BeautifulSoup(resp.text, 'lxml')
    html = etree.HTML(str(soup))
    
    # = = = = = = = = = = = = = = =

    json_ = json.loads(resp.text.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0].strip() + ']}]}')
    
    # = = = = = = = = = = = = = = =

    list_item_number += [w[2]['model']['action']['params']['itemId'].strip() if type(w[2]) == dict and 'model' in w[2] and 'action' in w[2]['model'] and 'params' in w[2]['model']['action'] and 'itemId' in w[2]['model']['action']['params'] else '' for w in json_['o']['w']]
    while '' in list_item_number:
        list_item_number.remove('')

    list_item_number = sorted(list(set(list_item_number)), key=list_item_number.index)
    
    # = = = = = = = = = = = = = = =

    i += 1

    # = = = = = = = = = = = = = = =

    if not html.xpath('//a[@type="next"]'):
        break

resp

1 1
1 2


<Response [200]>

In [6]:
len(list_item_number) == 0

False

In [7]:
output = pd.DataFrame({'No': [i+1 for i in range(len(list_item_number))],
                       'Item Number': list_item_number})

if url.startswith('https://www.ebay.com/'):
    output['Url'] = [f'https://www.ebay.com/itm/{item_number}?_ul=US&_stpos=91710&orig_cvip=true' for item_number in output['Item Number'].tolist()]
elif url.startswith('https://www.ebay.de/'):
    output['Url'] = [f'https://www.ebay.de/itm/{item_number}?_ul=DE&_stpos=10115&orig_cvip=true' for item_number in output['Item Number'].tolist()]
elif url.startswith('https://www.ebay.co.uk/'):
    output['Url'] = [f'https://www.ebay.co.uk/itm/{item_number}?_ul=GB&_stpos=BS81QU&orig_cvip=true' for item_number in output['Item Number'].tolist()]
elif url.startswith('https://www.ebay.com.au/'):
    output['Url'] = [f'https://www.ebay.com.au/itm/{item_number}?_ul=AU&_stpos=2019&orig_cvip=true' for item_number in output['Item Number'].tolist()]
elif url.startswith('https://www.ebay.ca/'):
    output['Url'] = [f'https://www.ebay.ca/itm/{item_number}?_ul=CA&_stpos=M5S2E8&orig_cvip=true' for item_number in output['Item Number'].tolist()]

output = output.sort_values(by=['No'],
                            ascending=[True],
                            ignore_index=True)

output.to_excel('./test_item_number.xlsx', index=False)

output

,No,Item Number,Url
0,1,235461387733,https://www.ebay.com/itm/235461387733?_ul=US&_...
1,2,233621258535,https://www.ebay.com/itm/233621258535?_ul=US&_...
2,3,334922305483,https://www.ebay.com/itm/334922305483?_ul=US&_...
3,4,255586148611,https://www.ebay.com/itm/255586148611?_ul=US&_...
4,5,234277033709,https://www.ebay.com/itm/234277033709?_ul=US&_...
...,...,...,...
413,414,335511427393,https://www.ebay.com/itm/335511427393?_ul=US&_...
414,415,234374652511,https://www.ebay.com/itm/234374652511?_ul=US&_...
415,416,335501268202,https://www.ebay.com/itm/335501268202?_ul=US&_...
416,417,256081676427,https://www.ebay.com/itm/256081676427?_ul=US&_...


In [8]:
crawler_status = 'ok'

crawler_status

'ok'

In [9]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [10]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[False],
                                            ignore_index=True)
    output_error.to_excel('./test_item_number_error.xlsx', index=False)

output_error

""
